In [1]:
import pickle
import tensorflow as tf
import numpy as np
import matrix_decompositions_tf as fctr

In [2]:
fid = open('iter_1074_weights.pkl','rb')
output = pickle.load(fid)
fid.close()
dictObj = fctr.dictionary_object2D_init(fftSz = output['fftSz'],D = output['D'],rho = output['rho'],objname = 'dictObj1',n_components = output['n_components'])

In [3]:
tf.reduce_max(tf.math.abs(tf.complex(dictObj.dhmul.Dfreal,dictObj.dhmul.Dfimag) - output['Dfprev']))

<tf.Tensor: shape=(), dtype=float64, numpy=7.29338156769531e-15>

In [4]:
tf.reduce_max(tf.math.abs(dictObj.qinv.L - output['L']))

<tf.Tensor: shape=(), dtype=float64, numpy=5.7731597291583e-15>

In [5]:
print(output['shift'])

tf.Tensor(1.8758866506294804e-13, shape=(), dtype=float64)


In [6]:
fid = open('iter_1075_weights.pkl','rb')
output2 = pickle.load(fid)
fid.close()
dictObj2 = fctr.dictionary_object2D_init(fftSz = output2['fftSz'],D = output2['D'],rho = output2['rho'],objname = 'dictObj2',n_components = output2['n_components'])

In [7]:
tf.reduce_max(tf.math.abs(tf.complex(dictObj2.dhmul.Dfreal,dictObj2.dhmul.Dfimag) - output2['Dfprev']))

<tf.Tensor: shape=(), dtype=float64, numpy=7.30071889589687e-15>

In [8]:
tf.reduce_max(tf.math.abs(dictObj2.qinv.L - output2['L']))

<tf.Tensor: shape=(), dtype=float64, numpy=5.773160419064828e-15>

In [9]:
print(output2['shift'])

tf.Tensor(1.7098299666192163e-13, shape=(), dtype=float64)


In [8]:
dictObj.dhmul.Dfreal = dictObj.dhmul.Dfreal.assign(dictObj2.dhmul.Dfreal)
dictObj.dhmul.Dfimag = dictObj.dhmul.Dfimag.assign(dictObj2.dhmul.Dfimag)
dictObj._dict_update()
y = dictObj.rho*dictObj.constx + dictObj.dhmul(dictObj.dmul(dictObj.constx))
z = dictObj.qinv(y)
shift = tf.math.abs(dictObj.constx - z)


In [13]:
print(shift.shape)

(5, 43, 22, 64, 1)


In [18]:
best_filter_shift = tf.reduce_min(shift,axis=1)
best_filter_shift = tf.reduce_min(best_filter_shift,axis=2)

In [19]:
tf.reduce_max(best_filter_shift)

<tf.Tensor: shape=(), dtype=float64, numpy=2.193878801300176e-09>

In [8]:
Dfdiff = tf.complex(dictObj2.dhmul.Dfreal,dictObj2.dhmul.Dfimag) - tf.complex(dictObj.dhmul.Dfreal,dictObj.dhmul.Dfimag)

In [9]:
tf.reduce_max(tf.math.abs(Dfdiff))

<tf.Tensor: shape=(), dtype=float64, numpy=6.222527893682096e-06>

In [16]:
import transforms as transf
U,V,approx = fctr.stack_svd(transf.ifft2d_inner(output['fftSz'])(Dfdiff),5,n_components=output['n_components'])

In [17]:
tf.reduce_max(tf.math.abs(approx - transf.ifft2d_inner(output['fftSz'])(Dfdiff)))

<tf.Tensor: shape=(), dtype=float64, numpy=8.04253943257098e-09>

In [9]:
print(dictObj.divide_by_R.R)
print(dictObj2.divide_by_R.R)

<tf.Variable 'dictObj1div_by_R/R:0' shape=(1, 1, 1, 1, 64) dtype=float64, numpy=
array([[[[[1.        , 1.        , 1.        , 1.00000005, 1.        ,
           1.        , 1.        , 1.        , 1.        , 1.        ,
           1.00000006, 0.99999998, 1.        , 1.        , 1.00000008,
           1.        , 1.        , 1.        , 1.        , 1.        ,
           1.00000007, 1.        , 1.        , 1.        , 1.        ,
           1.        , 1.        , 1.        , 1.        , 1.        ,
           1.        , 1.        , 1.        , 1.        , 1.        ,
           1.        , 1.        , 1.        , 1.        , 1.        ,
           1.        , 1.        , 1.        , 1.        , 1.        ,
           1.        , 1.        , 1.        , 1.        , 1.00000002,
           1.        , 1.00000003, 1.        , 1.        , 1.        ,
           1.        , 1.        , 1.        , 1.        , 1.        ,
           1.        , 1.        , 1.        , 1.        ]]]]])>
<t